In [ ]:
from sumatra import Client, APIClient
sumatra = Client('console.qa.sumatra.ai')
api = APIClient('console.qa.sumatra.ai')

In [ ]:
scowl = """
event test
name := $.name as string
greeting := 'Hello, ' + name
"""
sumatra.create_branch_from_scowl(scowl)
sumatra.publish_branch()

event = {'_type': 'test', 'name': 'World'}
response = api.send(event)
print(response)

In [ ]:
schemas = """
event user
user_id := $.user_id as string
age := $.age as int
uid := $.uid as string
phone := $.phone as string

event user_city
user_id := $.user_id as string
city_id := $.city_id as string

event city
city_id := $.city_id as string
city_tier := $.city_tier as int
"""
sumatra.create_branch_from_scowl(schemas)

In [ ]:
import pandas as pd
from faker import Faker
faker=Faker()

def gen_user():
  return {
      "user_id": faker.user_name(),
      "age": faker.random_int(13,99),
      "uid": faker.uuid4(),
      "phone": faker.phone_number(),
  }

def gen_city():
  return {
      "city_id": faker.city(),
      "city_tier": faker.random_int(1,3),
  }

user = pd.DataFrame([gen_user() for _ in range(25)])
city = pd.DataFrame([gen_city() for _ in range(8)])
user_city = pd.DataFrame({'user_id': user.user_id,
                          'city_id': city.sample(len(user), replace=True).city_id.to_list()})

sumatra.create_timeline_from_dataframes({'user': user, 'user_city': user_city, 'city': city},
                                        timeline='user_city_backfill')

In [ ]:
model_features = """
event query_model_features

user_id := $.user_id as string

age := Latest<user>(age by user_id)
phone := Latest<user>(phone by user_id)
city_id := Latest<user_city>(city_id by user_id)
city_tier := Latest<city>(city_tier by city_id)

-- Uncomment me
avg_age_by_tier_1h := Average(age by city_tier last hour)
users_per_city_2w := CountUnique(user_id by city_id last 2 weeks)
phone_has_extension := phone.Contains("x")
"""
sumatra.create_branch_from_scowl(schemas+model_features)

In [ ]:
queries = user.sample(10, replace=True)[['user_id']].reset_index(drop=True)
sumatra.create_timeline_from_dataframes({'query_model_features': queries},
                                        timeline='online_queries')

mtr = sumatra.materialize_many(['online_queries', 'user_city_backfill'])
enriched = mtr.get_enriched(event_type='query_model_features')
enriched